# HATLAS-SGP master catalogue

This notebook presents the merge of the various pristine catalogues to produce HELP mater catalogue on GAMA-09.

In [ ]:
from herschelhelp_internal import git_version
print("This notebook was run with herschelhelp_internal version: \n{}".format(git_version()))

In [ ]:
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))

import os
import time

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import Column, Table
import numpy as np
from pymoc import MOC

from herschelhelp_internal.masterlist import merge_catalogues, nb_merge_dist_plot, specz_merge
from herschelhelp_internal.utils import coords_to_hpidx, ebv, gen_help_id, inMoc

In [ ]:
TMP_DIR = os.environ.get('TMP_DIR', "./data_tmp")
OUT_DIR = os.environ.get('OUT_DIR', "./data")
SUFFIX = os.environ.get('SUFFIX', time.strftime("_%Y%m%d"))

try:
    os.makedirs(OUT_DIR)
except FileExistsError:
    pass

## I - Reading the prepared pristine catalogues

In [ ]:
atlas = Table.read("{}/ATLAS.fits".format(TMP_DIR))
kids = Table.read("{}/KIDS.fits".format(TMP_DIR))
ps1 = Table.read("{}/PS1.fits".format(TMP_DIR))
viking = Table.read("{}/VISTA-VIKING.fits".format(TMP_DIR))
des =  Table.read("{}/DES.fits".format(TMP_DIR))

## II - Merging tables

We first merge the optical catalogues and then add the infrared ones: CFHTLenS, CFHTLS, DECaLS, HSC, KIDS, PanSTARRS, UKIDSS-LAS, VISTA-VHS, and VISTA-VIKING.

At every step, we look at the distribution of the distances to the nearest source in the merged catalogue to determine the best crossmatching radius.

### ATLAS

In [ ]:
master_catalogue = atlas
master_catalogue['atlas_ra'].name = 'ra'
master_catalogue['atlas_dec'].name = 'dec'

### Add KIDS

In [ ]:
nb_merge_dist_plot(
    SkyCoord(master_catalogue['ra'], master_catalogue['dec']),
    SkyCoord(kids['kids_ra'], kids['kids_dec'])
)

In [ ]:
# Given the graph above, we use 0.8 arc-second radius
master_catalogue = merge_catalogues(master_catalogue, kids, "kids_ra", "kids_dec", radius=0.8*u.arcsec)

### Add PanSTARRS

In [ ]:
nb_merge_dist_plot(
    SkyCoord(master_catalogue['ra'], master_catalogue['dec']),
    SkyCoord(ps1['ps1_ra'], ps1['ps1_dec'])
)

In [ ]:
# Given the graph above, we use 0.8 arc-second radius
master_catalogue = merge_catalogues(master_catalogue, ps1, "ps1_ra", "ps1_dec", radius=0.8*u.arcsec)

### Add VIKING

In [ ]:
nb_merge_dist_plot(
    SkyCoord(master_catalogue['ra'], master_catalogue['dec']),
    SkyCoord(viking['viking_ra'], viking['viking_dec'])
)

In [ ]:
# Given the graph above, we use 1 arc-second radius
master_catalogue = merge_catalogues(master_catalogue, viking, "viking_ra", "viking_dec", radius=1.*u.arcsec)

### Add DES

In [ ]:
nb_merge_dist_plot(
    SkyCoord(master_catalogue['ra'], master_catalogue['dec']),
    SkyCoord(des['des_ra'], des['des_dec'])
)

In [ ]:
# Given the graph above, we use 0.8 arc-second radius
master_catalogue = merge_catalogues(master_catalogue, des, "des_ra", "des_dec", radius=0.8*u.arcsec)

### Cleaning

When we merge the catalogues, astropy masks the non-existent values (e.g. when a row comes only from a catalogue and has no counterparts in the other, the columns from the latest are masked for that row). We indicate to use NaN for masked values for floats columns, False for flag columns and -1 for ID columns.

In [ ]:
for col in master_catalogue.colnames:
    if "m_" in col or "merr_" in col or "f_" in col or "ferr_" in col or "stellarity" in col:
        master_catalogue[col].fill_value = np.nan
    elif "flag" in col:
        master_catalogue[col].fill_value = 0
    elif "id" in col:
        master_catalogue[col].fill_value = -1
        
master_catalogue = master_catalogue.filled()

In [ ]:
master_catalogue[:10].show_in_notebook()

## III - Merging flags and stellarity

Each pristine catalogue contains a flag indicating if the source was associated to a another nearby source that was removed during the cleaning process.  We merge these flags in a single one.

In [ ]:
flag_cleaned_columns = [column for column in master_catalogue.colnames
                        if 'flag_cleaned' in column]

flag_column = np.zeros(len(master_catalogue), dtype=bool)
for column in flag_cleaned_columns:
    flag_column |= master_catalogue[column]
    
master_catalogue.add_column(Column(data=flag_column, name="flag_cleaned"))
master_catalogue.remove_columns(flag_cleaned_columns)

Each pristine catalogue contains a flag indicating the probability of a source being a Gaia object (0: not a Gaia object, 1: possibly, 2: probably, 3: definitely).  We merge these flags taking the highest value.

In [ ]:
flag_gaia_columns = [column for column in master_catalogue.colnames
                     if 'flag_gaia' in column]

master_catalogue.add_column(Column(
    data=np.max([master_catalogue[column] for column in flag_gaia_columns], axis=0),
    name="flag_gaia"
))
master_catalogue.remove_columns(flag_gaia_columns)

Each prisitine catalogue may contain one or several stellarity columns indicating the probability (0 to 1) of each source being a star.  We merge these columns taking the highest value.

In [ ]:
stellarity_columns = [column for column in master_catalogue.colnames
                      if 'stellarity' in column]

print(", ".join(stellarity_columns))

In [ ]:

# We create an masked array with all the stellarities and get the maximum value, as well as its
# origin.  Some sources may not have an associated stellarity.
stellarity_array = np.array([master_catalogue[column] for column in stellarity_columns])
stellarity_array = np.ma.masked_array(stellarity_array, np.isnan(stellarity_array))

max_stellarity = np.max(stellarity_array, axis=0)
max_stellarity.fill_value = np.nan

no_stellarity_mask = max_stellarity.mask

master_catalogue.add_column(Column(data=max_stellarity.filled(), name="stellarity"))

stellarity_origin = np.full(len(master_catalogue), "NO_INFORMATION", dtype="S20")
stellarity_origin[~no_stellarity_mask] = np.array(stellarity_columns)[np.argmax(stellarity_array, axis=0)[~no_stellarity_mask]]

master_catalogue.add_column(Column(data=stellarity_origin, name="stellarity_origin"))

master_catalogue.remove_columns(stellarity_columns)

## IV - Adding E(B-V) column

In [ ]:
master_catalogue.add_column(
    ebv(master_catalogue['ra'], master_catalogue['dec'])
)

## V - Adding HELP unique identifiers and field columns

In [ ]:
master_catalogue.add_column(Column(gen_help_id(master_catalogue['ra'], master_catalogue['dec']),
                                   name="help_id"))
master_catalogue.add_column(Column(np.full(len(master_catalogue), "HATLAS-SGP", dtype='<U18'),
                                   name="field"))

In [ ]:
# Check that the HELP Ids are unique
if len(master_catalogue) != len(np.unique(master_catalogue['help_id'])):
    print("The HELP IDs are not unique!!!")
else:
    print("OK!")

## VI - Choosing between multiple values for the same filter

### ATLAS and KIDS
The ATLAS and KIDS surveys are both on the VLT Survey Telescope (VST). KIDS is significantly deeper so we take KIDS fluxes if available.

| Survey      | Bands observed   |
|-------------|------------------|
| ATLAS       | u, ul, griz      |
| KIDS        | ugri             |


In [ ]:
omegacam_origin = Table()
omegacam_origin.add_column(master_catalogue['help_id'])

In [ ]:
omegacam_stats = Table()
omegacam_stats.add_column(Column(data=['u', 'g','r','i'], name="Band"))
for col in ["ATLAS",  "KIDS"]:
    omegacam_stats.add_column(Column(data=np.full(4, 0), name="{}".format(col)))
    omegacam_stats.add_column(Column(data=np.full(4, 0), name="use {}".format(col)))
    omegacam_stats.add_column(Column(data=np.full(4, 0), name="{} ap".format(col)))
    omegacam_stats.add_column(Column(data=np.full(4, 0), name="use {} ap".format(col)))
    
#ul and z only in ATLAS so we can just rename those columns
for col in master_catalogue.colnames:
    if ('atlas_ul' in col) or ('atlas_z' in col):
        master_catalogue[col].name = col.replace('atlas', 'omegacam')

In [ ]:
omegacam_bands = ['u','g','r','i']
for band in omegacam_bands:

    # omegacam total flux 
    has_kids = ~np.isnan(master_catalogue['f_kids_' + band])
    has_atlas  = ~np.isnan(master_catalogue['f_atlas_' + band])

    

    use_kids = has_kids
    use_atlas = has_atlas & ~has_kids


    f_omegacam = np.full(len(master_catalogue), np.nan)
    f_omegacam[use_kids] = master_catalogue['f_kids_' + band][use_kids]
    f_omegacam[use_atlas] = master_catalogue['f_atlas_' + band][use_atlas]
  

    ferr_omegacam = np.full(len(master_catalogue), np.nan)
    ferr_omegacam[use_kids] = master_catalogue['ferr_kids_' + band][use_kids]
    ferr_omegacam[use_atlas] = master_catalogue['ferr_atlas_' + band][use_atlas]

    
    m_omegacam = np.full(len(master_catalogue), np.nan)
    m_omegacam[use_kids] = master_catalogue['m_kids_' + band][use_kids]
    m_omegacam[use_atlas] = master_catalogue['m_atlas_' + band][use_atlas]
   

    merr_omegacam = np.full(len(master_catalogue), np.nan)
    merr_omegacam[use_kids] = master_catalogue['merr_kids_' + band][use_kids]
    merr_omegacam[use_atlas] = master_catalogue['merr_atlas_' + band][use_atlas]


    flag_omegacam = np.full(len(master_catalogue), False, dtype=bool)
    flag_omegacam[use_kids] = master_catalogue['flag_kids_' + band][use_kids]
    flag_omegacam[use_atlas] = master_catalogue['flag_atlas_' + band][use_atlas]


    master_catalogue.add_column(Column(data=f_omegacam, name="f_omegacam_" + band))
    master_catalogue.add_column(Column(data=ferr_omegacam, name="ferr_omegacam_" + band))
    master_catalogue.add_column(Column(data=m_omegacam, name="m_omegacam_" + band))
    master_catalogue.add_column(Column(data=merr_omegacam, name="merr_omegacam_" + band))
    master_catalogue.add_column(Column(data=flag_omegacam, name="flag_omegacam_" + band))

    old_kids_columns = ['f_kids_' + band,
                               'ferr_kids_' + band,
                               'm_kids_' + band, 
                               'merr_kids_' + band,
                               'flag_kids_' + band]
    old_atlas_columns = ['f_atlas_' + band,
                               'ferr_atlas_' + band,
                               'm_atlas_' + band, 
                               'merr_atlas_' + band,
                               'flag_atlas_' + band]

    
    old_columns = old_kids_columns + old_atlas_columns 
    master_catalogue.remove_columns(old_columns)

    origin = np.full(len(master_catalogue), '     ', dtype='<U5')
    origin[use_kids] = "kids"
    origin[use_atlas] = "atlas"
   
    
    omegacam_origin.add_column(Column(data=origin, name= 'f_omegacam_' + band ))
    
  
    # omegacam aperture flux 
    has_ap_kids = ~np.isnan(master_catalogue['f_ap_kids_' + band])
    has_ap_atlas  = ~np.isnan(master_catalogue['f_ap_atlas_' + band])
  
    

    use_ap_kids = has_ap_kids
    use_ap_atlas = has_ap_atlas & ~has_ap_kids


    f_ap_omegacam = np.full(len(master_catalogue), np.nan)
    f_ap_omegacam[use_ap_kids] = master_catalogue['f_ap_kids_' + band][use_ap_kids]
    f_ap_omegacam[use_ap_atlas] = master_catalogue['f_ap_atlas_' + band][use_ap_atlas]
  

    ferr_ap_omegacam = np.full(len(master_catalogue), np.nan)
    ferr_ap_omegacam[use_ap_kids] = master_catalogue['ferr_ap_kids_' + band][use_ap_kids]
    ferr_ap_omegacam[use_ap_atlas] = master_catalogue['ferr_ap_atlas_' + band][use_ap_atlas]
   
    
    m_ap_omegacam = np.full(len(master_catalogue), np.nan)
    m_ap_omegacam[use_ap_kids] = master_catalogue['m_ap_kids_' + band][use_ap_kids]
    m_ap_omegacam[use_ap_atlas] = master_catalogue['m_ap_atlas_' + band][use_ap_atlas]
   

    merr_ap_omegacam = np.full(len(master_catalogue), np.nan)
    merr_ap_omegacam[use_ap_kids] = master_catalogue['merr_ap_kids_' + band][use_ap_kids]
    merr_ap_omegacam[use_ap_atlas] = master_catalogue['merr_ap_atlas_' + band][use_ap_atlas]



    master_catalogue.add_column(Column(data=f_ap_omegacam, name="f_ap_omegacam_" + band))
    master_catalogue.add_column(Column(data=ferr_ap_omegacam, name="ferr_ap_omegacam_" + band))
    master_catalogue.add_column(Column(data=m_ap_omegacam, name="m_ap_omegacam_" + band))
    master_catalogue.add_column(Column(data=merr_ap_omegacam, name="merr_ap_omegacam_" + band))


    old_ap_kids_columns = ['f_ap_kids_' + band,
                               'ferr_ap_kids_' + band,
                               'm_ap_kids_' + band, 
                               'merr_ap_kids_' + band]
    old_ap_atlas_columns = ['f_ap_atlas_' + band,
                               'ferr_ap_atlas_' + band,
                               'm_ap_atlas_' + band, 
                               'merr_ap_atlas_' + band]

    
    old_ap_columns = old_ap_kids_columns + old_ap_atlas_columns 
    master_catalogue.remove_columns(old_ap_columns)

    origin_ap = np.full(len(master_catalogue), '     ', dtype='<U5')
    origin_ap[use_ap_kids] = "kids"
    origin_ap[use_ap_atlas] = "atlas"
  
    
    omegacam_origin.add_column(Column(data=origin_ap, name= 'f_ap_omegacam_' + band ))

   
    omegacam_stats['KIDS'][omegacam_stats['Band'] == band] = np.sum(has_kids)
    omegacam_stats['ATLAS'][omegacam_stats['Band'] == band] = np.sum(has_atlas)

    omegacam_stats['use KIDS'][omegacam_stats['Band'] == band] = np.sum(use_kids)
    omegacam_stats['use ATLAS'][omegacam_stats['Band'] == band] = np.sum(use_atlas)

    omegacam_stats['KIDS ap'][omegacam_stats['Band'] == band] = np.sum(has_ap_kids)
    omegacam_stats['ATLAS ap'][omegacam_stats['Band'] == band] = np.sum(has_ap_atlas)
   
    omegacam_stats['use KIDS ap'][omegacam_stats['Band'] == band] = np.sum(use_ap_kids)
    omegacam_stats['use ATLAS ap'][omegacam_stats['Band'] == band] = np.sum(use_ap_atlas)

In [ ]:
omegacam_stats.show_in_notebook()

In [ ]:
omegacam_origin.write("{}/hatlas-sgp_omegacam_fluxes_origins{}.fits".format(OUT_DIR, SUFFIX))

## VII.a Wavelength domain coverage

We add a binary `flag_optnir_obs` indicating that a source was observed in a given wavelength domain:

- 1 for observation in optical;
- 2 for observation in near-infrared;
- 4 for observation in mid-infrared (IRAC).

It's an integer binary flag, so a source observed both in optical and near-infrared by not in mid-infrared would have this flag at 1 + 2 = 3.

*Note 1: The observation flag is based on the creation of multi-order coverage maps from the catalogues, this may not be accurate, especially on the edges of the coverage.*

*Note 2: Being on the observation coverage does not mean having fluxes in that wavelength domain. For sources observed in one domain but having no flux in it, one must take into consideration de different depths in the catalogue we are using.*

In [ ]:
atlas_moc = MOC(filename="../../dmu0/dmu0_ATLAS/data/ATLAS_HATLAS-SGP_MOC.fits")
kids_moc = MOC(filename="../../dmu0/dmu0_KIDS/data/KIDS-DR3_HATLAS-SGP_MOC.fits")
ps1_moc = MOC(filename="../../dmu0/dmu0_PanSTARRS1-3SS/data/PanSTARRS1-3SS_HATLAS-SGP_MOC.fits")
viking_moc = MOC(filename="../../dmu0/dmu0_VISTA-VIKING/data/VIKING_SGP_MOC.fits")
des_moc = MOC(filename="../../dmu0/dmu0_DES/data/DES-DR1_HATLAS-SGP_MOC.fits")

In [ ]:
was_observed_optical = inMoc(
    master_catalogue['ra'], master_catalogue['dec'],
    atlas_moc + kids_moc + ps1_moc + des_moc) 

was_observed_nir = inMoc(
    master_catalogue['ra'], master_catalogue['dec'],
    viking_moc
)

was_observed_mir = np.zeros(len(master_catalogue), dtype=bool)

#was_observed_mir = inMoc(
#    master_catalogue['ra'], master_catalogue['dec'],   
#)

In [ ]:
master_catalogue.add_column(
    Column(
        1 * was_observed_optical + 2 * was_observed_nir + 4 * was_observed_mir,
        name="flag_optnir_obs")
)

## VII.b Wavelength domain detection

We add a binary `flag_optnir_det` indicating that a source was detected in a given wavelength domain:

- 1 for detection in optical;
- 2 for detection in near-infrared;
- 4 for detection in mid-infrared (IRAC).

It's an integer binary flag, so a source detected both in optical and near-infrared by not in mid-infrared would have this flag at 1 + 2 = 3.

*Note 1: We use the total flux columns to know if the source has flux, in some catalogues, we may have aperture flux and no total flux.*

To get rid of artefacts (chip edges, star flares, etc.) we consider that a source is detected in one wavelength domain when it has a flux value in **at least two bands**. That means that good sources will be excluded from this flag when they are on the coverage of only one band.

In [ ]:
# SpARCS is a catalogue of sources detected in r (with fluxes measured at 
# this prior position in the other bands).  Thus, we are only using the r
# CFHT band.
# Check to use catalogue flags from HSC and PanSTARRS.
nb_optical_flux = (
    #Omegacam 
    1 * ~np.isnan(master_catalogue['f_omegacam_ul']) +
    1 * ~np.isnan(master_catalogue['f_omegacam_u']) +
    1 * ~np.isnan(master_catalogue['f_omegacam_g']) +
    1 * ~np.isnan(master_catalogue['f_omegacam_r']) +
    1 * ~np.isnan(master_catalogue['f_omegacam_i']) +
    1 * ~np.isnan(master_catalogue['f_omegacam_z']) +
    # PanSTARRS
    1 * ~np.isnan(master_catalogue['f_gpc1_g']) +
    1 * ~np.isnan(master_catalogue['f_gpc1_r']) +
    1 * ~np.isnan(master_catalogue['f_gpc1_i']) +
    1 * ~np.isnan(master_catalogue['f_gpc1_z']) +
    1 * ~np.isnan(master_catalogue['f_gpc1_y']) +
    #DES
    1 * ~np.isnan(master_catalogue['f_decam_g']) +
    1 * ~np.isnan(master_catalogue['f_decam_r']) +    
    1 * ~np.isnan(master_catalogue['f_decam_i']) +
    1 * ~np.isnan(master_catalogue['f_decam_z']) +
    1 * ~np.isnan(master_catalogue['f_decam_y']) 
)

nb_nir_flux = (
    1 * ~np.isnan(master_catalogue['f_vista_z']) +
    1 * ~np.isnan(master_catalogue['f_vista_y']) +
    1 * ~np.isnan(master_catalogue['f_vista_j']) +
    1 * ~np.isnan(master_catalogue['f_vista_h']) +
    1 * ~np.isnan(master_catalogue['f_vista_k'])
)

nb_mir_flux = np.zeros(len(master_catalogue), dtype=bool)

In [ ]:
has_optical_flux = nb_optical_flux >= 2
has_nir_flux = nb_nir_flux >= 2
has_mir_flux = nb_mir_flux >= 2

master_catalogue.add_column(
    Column(
        1 * has_optical_flux + 2 * has_nir_flux + 4 * has_mir_flux,
        name="flag_optnir_det")
)

## VIII - Cross-identification table

We are producing a table associating to each HELP identifier, the identifiers of the sources in the pristine catalogue. This can be used to easily get additional information from them.


In [ ]:
master_catalogue['help_id', 'atlas_id',
                 'kids_id', 'ps1_id', 'viking_id', 'des_id'].write(
    "{}/master_list_cross_ident_hatlas-sgp{}.fits".format(OUT_DIR, SUFFIX))
master_catalogue.remove_columns(['atlas_id', 'kids_id',
                                 'ps1_id', 'viking_id', 'des_id'])

## IX - Adding HEALPix index

We are adding a column with a HEALPix index at order 13 associated with each source.

In [ ]:
master_catalogue.add_column(Column(
    data=coords_to_hpidx(master_catalogue['ra'], master_catalogue['dec'], order=13),
    name="hp_idx"
))

## X - Saving the catalogue

In [ ]:
columns = ["help_id", "field", "ra", "dec", "hp_idx"]

bands = [column[5:] for column in master_catalogue.colnames if 'f_ap' in column]
for band in bands:
    columns += ["f_ap_{}".format(band), "ferr_ap_{}".format(band),
                "m_ap_{}".format(band), "merr_ap_{}".format(band),
                "f_{}".format(band), "ferr_{}".format(band),
                "m_{}".format(band), "merr_{}".format(band),
                "flag_{}".format(band)]    
    
columns += ["stellarity", "flag_cleaned", "flag_merged", "flag_gaia", "flag_optnir_obs", "flag_optnir_det", "ebv"]

In [ ]:
# We check for columns in the master catalogue that we will not save to disk.
print("Missing columns: {}".format(set(master_catalogue.colnames) - set(columns)))

In [ ]:
master_catalogue[columns].write("{}/master_catalogue_hatlas-sgp{}.fits".format(OUT_DIR, SUFFIX))